In [0]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle

mv: cannot stat 'kaggle.json': No such file or directory


In [4]:
!kaggle competitions download -c textract-2019

textract_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
textract_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
textract_sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip textract_train.csv.zip
!unzip textract_test.csv.zip

Archive:  textract_train.csv.zip
replace textract_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
df = pd.read_csv('textract_train.csv')

In [0]:
sentences = df['Contents']
labels = df['Label']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42, stratify = labels)

In [0]:
mst_train = []
mst_test = []
import re
def counting_Mistakes2(text):  
    return len(re.findall(r"\.[^\s-]", text))

for sentence in X_train:
  mst_train.append(counting_Mistakes2(sentence))

for sentence in X_test:
  mst_test.append(counting_Mistakes2(sentence))

In [0]:
def clean_word(sample):
    to_replace = """[,\.!?'"123456789]-():;”"""
    for char in to_replace:
        sample = sample.replace(char,'')
    return sample

In [0]:
new_train = []
for article in tqdm(X_train):
    new_article = []
    for word in article.split(' '):
        new_word = clean_word(word)
        if new_word != '' and new_word != ' ':
            new_article.append(new_word)
        if (word.find('.') != -1): 
          new_article.append('.')
        if (word.find(',') != -1): 
          new_article.append(',')
        if (word.find('”') != -1):
          new_article.append('”')
    new_train.append(new_article)

In [0]:
new_test = []
for article in tqdm(X_test):
    new_article = []
    for word in article.split(' '):
        new_word = clean_word(word)
        if new_word != '' and new_word != ' ':
            new_article.append(new_word)
    new_test.append(new_article)

In [0]:
from gensim.models import Word2Vec
w2v_size = 300
w2v_model = Word2Vec(min_count=100,
                     window=4,
                     size=w2v_size,
                     workers=2)
w2v_model.build_vocab(new_train)
w2v_model.train(new_train, total_examples=w2v_model.corpus_count, epochs=30)

In [0]:
# vectorizing train data
X_train = new_train
X_dev = new_test
padding= np.zeros(w2v_size)
X_train_vec = []
dim = 200
for sample in X_train:
    vec = []
    k = 0
    for word in sample:
        if k >= dim:
            break
        k += 1
        try:
            vec.append(w2v_model.wv[word])
        except:
            vec.append(padding)
    while k < dim:
        vec.append(padding)
        k += 1
    X_train_vec.append(np.array(vec))
X_train_vec = np.array(X_train_vec)

# vectorizing dev data
padding= np.zeros(w2v_size)
X_dev_vec = []
for sample in X_dev:
    vec = []
    k = 0
    for word in sample:
        if k >= dim:
            break
        k += 1
        try:
            vec.append(w2v_model.wv[word])
        except:
            vec.append(padding)
    while k < dim:
        vec.append(padding)
        k += 1
    X_dev_vec.append(np.array(vec))
X_dev_vec = np.array(X_dev_vec)

In [0]:
X_dev = X_dev_vec
X_train = X_train_vec
y_dev = y_test
y_train = np.array(list(y_train))
y_dev = np.array(list(y_dev))
mst_train = np.array(list(mst_train))
mst_test = np.array(list(mst_test))
print(X_train.shape)
print(X_dev.shape)
print(y_train.shape)
print(y_dev.shape)
print(mst_train.shape)
print(mst_test.shape)

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Bidirectional, GlobalMaxPooling1D, Conv1D, Dropout, MaxPool1D,Input,ThresholdedReLU
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, Callback, EarlyStopping

In [0]:
inputs = Input(shape=(200,300))

x = Conv1D(256, 7,use_bias = False, padding='same')(inputs)
x = ThresholdedReLU(theta=0.75)(x)
x = Conv1D(128, 5, padding = 'same')(x)
x = ThresholdedReLU(theta=0.75)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.25)(x)
conv_output = Dense(32, activation = 'relu')(x)

seq_lenght_input = Input(shape = (1,), name = 'seq_lenght_input')



x = keras.layers.concatenate([conv_output, seq_lenght_input])
x = Dense(32,activation = 'relu')(x)
x = Dense(16,activation = 'relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[inputs, seq_lenght_input] , outputs=predictions)

model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy'])

In [0]:

mcp_save = ModelCheckpoint('tempMdlml.hdf5', save_best_only=True, monitor='val_loss', mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
csv_logger = CSVLogger('log2.csv', append=True, separator=';')

In [0]:
model.fit([X_train, mst_train],y_train, epochs = 2000, validation_data=([X_dev, mst_test],y_dev), batch_size = 32,callbacks= [mcp_save, earlyStopping, csv_logger])